Importing needed libraries

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Getting a BS to parse a wiki page and saving BS object to 'soup' variable

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

Saving column names and table content from parsed wiki page to the lists

In [59]:
data_table = soup.find('table')
name = data_table.find_all('th')
column_names = [''.join(string.text.split()) for string in name]
rows = data_table.find_all('tr')
data_list = []
for row in rows:
    rowlist = []
    try:
        tds = row.find_all('td')
        if len(tds) == 0: continue
        for td in tds:
            rowlist.append(td.text.strip())
        data_list.append(rowlist)
    except:
        pass

Creating a dataframe and cleaning data according to assignment instructions

In [61]:
df = pd.DataFrame(data=data_list, columns=column_names)
df = df.drop(df[df.Borough == 'Not assigned'].index)
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking a shape of cleaned data frame

In [62]:
df.shape

(103, 3)